![data-x](https://raw.githubusercontent.com/afo/data-x-plaksha/master/imgsource/dx_logo.png)

---
# Notebook: SQL in Python

**Author list:** Alexander Fred Ojala & Sindhuja Jeyabal

**References / Sources:** 


**License Agreement:** Feel free to do whatever you want with this code

___

# Table of Contents

### * [Basics: sqlite3 (Python builtin)](#sec1)
### * [Pandas and SQL](#sec2)
### * [Breakout](#sec3)
### * [ORM: SQLAlchemy](#sec4)

<div id="sec1"></div>

## SQLite 3

In [ ]:
# sqlite3 package comes with the Python installation
import sqlite3

In [ ]:
# list files in w.d.
!ls

In [ ]:
# open connnection to a db file stored locally on disk
# if file doesn't exist it is created
connection = sqlite3.connect('company.db')

In [ ]:
!ls # file is created

In [ ]:
# In order to run SQL commands with
# sqlite 3 we must create a cursor object
# that traverses the database
cursor = connection.cursor()

# to run sql commands execute them

In [ ]:
# Check that we are working with an empty db
cursor.execute("DROP TABLE IF EXISTS employee;")

In [ ]:
# We can define long SQL commands within three quotes

sql_command = """
CREATE TABLE employee (
staff_number INTEGER PRIMARY KEY,
fname VARCHAR(20),
lname VARCHAR(30),
gender CHAR(1),
joining DATE,
birth_date DATE);"""

# VARCHAR string that can be up to X characters
# CHAR has to be X characters

**Link to SQL datatypes:** [https://www.w3schools.com/sql/sql_datatypes.asp](https://www.w3schools.com/sql/sql_datatypes.asp)

In [ ]:
# In order to run SQL command on the databse file
# we have to execute them with the cursor
cursor.execute(sql_command)

In [ ]:
sql_command = '''
INSERT INTO employee
(staff_number, fname, lname, gender, birth_date)
VALUES
(NULL, "William", "Shakespeare", "m", "1961-10-25");'''

cursor.execute(sql_command)

In [ ]:
sql_command = '''
INSERT INTO employee 
(staff_number, fname, lname, gender, birth_date) 
VALUES (NULL, "Frank", "Schiller", "m", "1955-08-17");'''


cursor.execute(sql_command)

In [ ]:
# never forget to commit your queries
# if you have updated the database
# and close the connection when you're done

connection.commit() 

connection.close()

In [ ]:
# reopen the connection and create cursor
connection = sqlite3.connect('company.db')

cursor = connection.cursor()

In [ ]:
a = cursor.execute('SELECT * FROM employee;')

In [ ]:
# fetch values, a.fetchall is a generator object
for row in a.fetchall():
    print(row)

In [ ]:
# close connection
connection.close()

In [ ]:
# Assume we already have a company.db sqlite database file
# with a Table called employees

# Add several values with sqlite3 and string formatting

connection = sqlite3.connect("company.db")

cursor = connection.cursor()

# list of tuples we want to add to the database
staff_data = [ ("Alfred", "Nobel", "m", "1921-05-02"), \
              ("Klaus", "Kinski", "m", "1935-09-07"),  \
              ("Jane", "Wall", "f", "1989-05-03") ]

for p in staff_data:
    
    # define string to be formatted, name format placeholders within curly brackets
    format_str = '''
    INSERT INTO employee (staff_number, fname, lname, gender, birth_date)    
    VALUES (NULL, "{first}", "{last}", "{gender}", "{birthdate}");
    '''

    # define SQL command
    sql_command = format_str.format(first=p[0], last=p[1], 
                                gender=p[2], birthdate = p[3]) 
    
    cursor.execute(sql_command)
    
# Commit changes
connection.commit()

In [ ]:
# Check that the changes were committed
cursor.execute("SELECT * FROM employee") 

print("fetchall:") 

result = cursor.fetchall() 

for r in result:

    print(r) 

In [ ]:
# Alternative approach

# Count rows in the database
n_rows = cursor.execute('SELECT Count(*) FROM employee').fetchone()[0]

res = cursor.execute("SELECT * FROM employee") 

print("\nFetch one at a time:") 
for i in range(0,n_rows): # we need to unpack
    tmp = res.fetchone() 

    print(tmp)

<div id="sec2"></div>

# Quick Data Analysis with SQL and Pandas

### Preferred method when running SELECT statements

In [ ]:
import pandas as pd
import datetime as dt 

In [ ]:
# this gives the output as a dataframe
pd.read_sql_query('SELECT * FROM employee',con = connection)

#### Initialize database with filename 311_NYC.db in current working directory

In [ ]:
!head -n10 311_NYC.csv

In [ ]:
!wc -l 311_NYC.csv #number of lines

In [ ]:
connection = sqlite3.connect('311_NYC.db')

### Read in big CSV file to Database

Original data set can be downloaded here (several GB's): https://nycopendata.socrata.com/Social-Services/311-Service-Requests-from-2010-to-Present/erm2-nwe9

In [ ]:
start = dt.datetime.now()
chunksize = 20000
j = 0
index_start = 1

for df in pd.read_csv('311_NYC.csv', chunksize=chunksize, iterator=True, encoding='utf-8'):
    
    df = df.rename(columns={c: c.replace(' ', '') for c in df.columns}) # Remove spaces from columns

    df['CreatedDate'] = pd.to_datetime(df['CreatedDate']) # Convert to datetimes
    df['ClosedDate'] = pd.to_datetime(df['ClosedDate'])

    df.index += index_start

    # Columns to keep
    columns = ['Agency', 'CreatedDate', 'ClosedDate', 'ComplaintType', 'Descriptor',
               'TimeToCompletion','City']

    for c in df.columns:
        if c not in columns:
            df = df.drop(c, axis=1)    

    
    j+=1

    df.to_sql('data', connection, if_exists='append') # name of SQL table, connection, append
    
    
    print('{} seconds: completed {} rows'.format((dt.datetime.now() - start).seconds, j*chunksize))
    
    index_start = df.index[-1] + 1 # update index start

#### Preview the DataBase

In [ ]:
pd.read_sql_query('SELECT * FROM data', connection).head()

#### Select a few columns

In [ ]:
pd.read_sql_query('SELECT Agency, Descriptor FROM data LIMIT 3', connection).head()

#### Filter rows using WHERE

In [ ]:
# we can also save the output as a DataFrame
df = pd.read_sql_query('SELECT ComplaintType, Descriptor, Agency '
                       'FROM data '
                       'WHERE Agency = "DOT" '
                       'LIMIT 10', connection)
df

#### Filter multiple values in a column using IN

In [ ]:
df = pd.read_sql_query('SELECT ComplaintType, Descriptor, Agency '
                       'FROM data '
                       'WHERE Agency IN ("NYPD", "DOB")'
                       'LIMIT 20', connection)
df.head()

#### Find Unique values using DISTINCT

In [ ]:
df = pd.read_sql_query('SELECT DISTINCT Agency FROM data', connection)
df

####  Query values using Count(*) and GROUP BY and ORDER BY

#### Which agency receives the most complaints?

In [ ]:
sql_command = '''
SELECT Agency, Count(*) 
AS num_complaints 
FROM data 
GROUP BY Agency 
ORDER BY num_complaints DESC'''

pd.read_sql_query(sql_command,connection)

#### What is the most common complaint type?

In [ ]:
df = pd.read_sql_query('SELECT ComplaintType, Count(*) AS num_complaints '
                       'FROM data '
                       'GROUP BY ComplaintType '
                       'ORDER BY num_complaints DESC', connection)

df.head()

#### Number of complaints per city for the top 10 cities with most complaints

In [ ]:
sql_command = '''
SELECT City, Count(*) AS num_complaints
FROM Data
GROUP BY City
ORDER BY num_complaints DESC
LIMIT 10
'''

df = pd.read_sql_query(sql_command, connection)
df

#### Perform Case Insensitive Queries

In [ ]:
sql_command = '''
SELECT City, Count(*) AS num_complaints
FROM Data
GROUP BY City COLLATE NOCASE
ORDER BY num_complaints DESC
LIMIT 10
'''

df = pd.read_sql_query(sql_command, connection)
df # Jamaica, Flushing etc have more complaints now

<div id="sec3"></div>

# Breakout Problems

# INTRODUCTION TO SQL

In this intro to SQL you can either use an SQL online editor (or sqlite3 with pandas in this notebook): 

### [https://www.w3schools.com/sql/trysql.asp?filename=trysql_select_all](https://www.w3schools.com/sql/trysql.asp?filename=trysql_select_all)

There are already some tables in the online Database, namely: 

	Categories, Employees, OrderDetails	, Orders, Products, Shippers, and Suppliers. 

We are not going to use them, and if you want you can drop them by running `DROP TABLE [table-name];`



## Exercises: Let's play with Dogs (& SQL)

First create a table called parents. It has two columns: 'parent' and 'child'. The first column indicates the parent of the child in the second column. We will use a new form of `CREATE TABLE` expression to produce this table.

	CREATE TABLE parents AS
	  SELECT "abraham" AS parent, "barack" AS child UNION
	  SELECT "abraham",           "clinton"         UNION
	  SELECT "delano",            "herbert"         UNION
	  SELECT "fillmore",          "abraham"         UNION
	  SELECT "fillmore",          "delano"          UNION
	  SELECT "fillmore",          "grover"          UNION
	  SELECT "eisenhower",        "fillmore";
	  

### Picture of the Dog Family Tree (illustration of parents table)

(A = abrham, B = barack, etc.)

<center><img src="https://github.com/alexanderfo/data-x_public/raw/master/L13_SQL/imgs/family_tree.png" width="200" /></center>
	  
## Q1 Simple SELECTS (on the parents table)
1. SELECT all records in the table.
2. SELECT child and parent, where abraham is the parent.
3. SELECT all children that have an 'e' in their name (hint: use LIKE and '%e%').
4. SELECT all unique parents (use SELECT DISTINCT) and order them by name, descending order (i.e. fillmore first)
5. **Difficult***: SELECT all dogs that are siblings (one-to-one relations). Only show a sibling pair once. To do this you need to select two times from the parents table.

## Q2 Joins

Create a new table called dogs, which indicates the fur type of every dog. In the image above: long haired dogs = red dashed box, curly haired dogs = black fluffy box, and short haired dogs = grey dotted box. 

Create the table by running:
	
	CREATE TABLE dogs AS
	  SELECT "abraham" AS name, "long" AS fur UNION
	  SELECT "barack",          "short"       UNION
	  SELECT "clinton",         "long"        UNION
	  SELECT "delano",          "long"        UNION
	  SELECT "eisenhower",      "short"       UNION
	  SELECT "fillmore",        "curly"       UNION
	  SELECT "grover",          "short"       UNION
	  SELECT "herbert",         "curly";

1. COUNT the number of short haired dogs
2. JOIN tables parents and dogs and SELECT the parents of curly dogs.
2. **Difficult**: JOIN tables parents and dogs, and SELECT the parents and children that have the same fur type. Only show them once.


## Q3 Aggregate functions, numerical logic and grouping

Create a new table with many different animals. The table includes the animal's kind, number of legs and weight. Create it by running:

	create table animals as
	 select "dog" as kind, 4 as legs, 20 as weight union
	 select "cat" , 4 , 10 union
	 select "ferret" , 4 , 10 union
	 select "parrot" , 2 , 6 union
	 select "penguin" , 2 , 10 union
	select "t-rex" , 2 , 12000;
	
1. SELECT the animal with the minimum weight. Display kind and min_weight.
2. Use aggregate function AVG to display a table with the average number of legs and the average weight.
3. SELECT the animal kind(s) that have more than two legs, but weighs less than 20. Display kind, weight, legs.
4. SELECT the average weight for all the animals with 2 legs and the animals with 4 legs (by using GROUP BY).

.


*These exercises are inspired by the Lectures in CS61A (Fall 2014).*


In [ ]:
# You can also solve it in Python using sqlite3 and pandas

import sqlite3

connection = sqlite3.connect('breakout.db')

cursor = connection.cursor()

In [ ]:
cursor.execute('DROP TABLE IF EXISTS parents')

In [ ]:
# Create the table
sql_command = '''
CREATE TABLE parents AS
  SELECT "abraham" AS parent, "barack" AS child UNION
  SELECT "abraham",           "clinton"         UNION
  SELECT "delano",            "herbert"         UNION
  SELECT "fillmore",          "abraham"         UNION
  SELECT "fillmore",          "delano"          UNION
  SELECT "fillmore",          "grover"          UNION
  SELECT "eisenhower",        "fillmore";
'''

cursor.execute(sql_command)

In [ ]:
connection.commit()

# Q1 Simple SELECTS (on the parents table)
* 1: SELECT all records in the table.
* 2: SELECT child and parent, where abraham is the parent.
* 3: SELECT all children that have an 'e' in their name (hint: use LIKE and '%e%').
* 4: SELECT all unique parents (use SELECT DISTINCT) and order them by name, descending order (i.e. fillmore first)
* 5: **Difficult***: SELECT all dogs that are siblings (one-to-one relations). Only show a sibling pair once. To do this you need to select two times from the parents table.

In [ ]:
# 1.
pd.read_sql_query('SELECT * FROM parents;', connection)

<div id="sec4"></div>

## Q2 Joins

Create a new table called dogs, which indicates the fur type of every dog. In the image above: long haired dogs = red dashed box, curly haired dogs = black fluffy box, and short haired dogs = grey dotted box. 

Create the table by running:
	
	CREATE TABLE dogs AS
	  SELECT "abraham" AS name, "long" AS fur UNION
	  SELECT "barack",          "short"       UNION
	  SELECT "clinton",         "long"        UNION
	  SELECT "delano",          "long"        UNION
	  SELECT "eisenhower",      "short"       UNION
	  SELECT "fillmore",        "curly"       UNION
	  SELECT "grover",          "short"       UNION
	  SELECT "herbert",         "curly";

1. COUNT the number of short haired dogs
2. JOIN tables parents and dogs and SELECT the parents of curly dogs.
2. **Difficult**: JOIN tables parents and dogs, and SELECT the parents and children that have the same fur type. Only show them once.

In [ ]:
sql_command = '''
CREATE TABLE dogs AS
  SELECT "abraham" AS name, "long" AS fur UNION
  SELECT "barack",          "short"       UNION
  SELECT "clinton",         "long"        UNION
  SELECT "delano",          "long"        UNION
  SELECT "eisenhower",      "short"       UNION
  SELECT "fillmore",        "curly"       UNION
  SELECT "grover",          "short"       UNION
  SELECT "herbert",         "curly";
'''
cursor.execute(sql_command)

In [ ]:
connection.commit()

In [ ]:
# 1.
pd.read_sql_query('SELECT Count(*) FROM dogs WHERE fur="short"',connection)


## Q3 Aggregate functions, numerical logic and grouping

Create a new table with many different animals. The table includes the animal's kind, number of legs and weight. Create it by running:

	create table animals as
	 select "dog" as kind, 4 as legs, 20 as weight union
	 select "cat" , 4 , 10 union
	 select "ferret" , 4 , 10 union
	 select "parrot" , 2 , 6 union
	 select "penguin" , 2 , 10 union
	select "t-rex" , 2 , 12000;
	
1. SELECT the animal with the minimum weight. Display kind and min_weight.
2. Use aggregate function AVG to display a table with the average number of legs and the average weight.
3. SELECT the animal kind(s) that have more than two legs, but weighs less than 20. Display kind, weight, legs.
4. SELECT the average weight for all the animals with 2 legs and the animals with 4 legs (by using GROUP BY).

.

In [ ]:
sql_command = '''
create table animals as
 select "dog" as kind, 4 as legs, 20 as weight union
 select "cat" , 4 , 10 union
 select "ferret" , 4 , 10 union
 select "parrot" , 2 , 6 union
 select "penguin" , 2 , 10 union
select "t-rex" , 2 , 12000;
'''
cursor.execute(sql_command)
connection.commit()

In [ ]:
# 1.
sql_command = '''
SELECT kind, MIN(weight)
FROM animals;
'''
pd.read_sql_query(sql_command,connection)

# ORM: Object Relational Mapper

# Use SQLAlchemy to define tables as classes
### Make use of ORM (Object Relational Mapper)
Great for large scale systems

In [ ]:
import os
import sys
from sqlalchemy import Column, ForeignKey, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship
from sqlalchemy import create_engine
 
Base = declarative_base()
 
class Person(Base):
    __tablename__ = 'person'
    # Here we define columns for the table person
    # Notice that each column is also a normal Python instance attribute.
    id = Column(Integer, primary_key=True)
    name = Column(String(250), nullable=False)
    
class Address(Base):
    __tablename__ = 'address'
    # Here we define columns for the table address.
    # Notice that each column is also a normal Python instance attribute.
    id = Column(Integer, primary_key=True)
    street_name = Column(String(250))
    street_number = Column(String(250))
    post_code = Column(String(250), nullable=False)
    person_id = Column(Integer, ForeignKey('person.id'))
    person = relationship(Person)
    
    
# Create an engine that stores data in the local directory's
# sqlalchemy_example.db file.
engine = create_engine('sqlite:///sqlalchemy_example.db')
 
# Create all tables in the engine. This is equivalent to "Create Table"
# statements in raw SQL.
Base.metadata.create_all(engine)

In [ ]:
from sqlalchemy.orm import sessionmaker
 
# Bind the engine to the metadata of the Base class so that the
# declaratives can be accessed through a DBSession instance
Base.metadata.bind = engine
 
DBSession = sessionmaker(bind=engine)
# A DBSession() instance establishes all conversations with the database
# and represents a "staging zone" for all the objects loaded into the
# database session object. Any change made against the objects in the
# session won't be persisted into the database until you call
# session.commit(). If you're not happy about the changes, you can
# revert all of them back to the last commit by calling
# session.rollback()
session = DBSession()
 
# Insert a Person in the person table
new_person = Person(name='new person')
session.add(new_person)
session.commit()
 
# Insert an Address in the address table
new_address = Address(post_code='00000', person=new_person,
                     street_name='28th st.')
session.add(new_address)
session.commit()

In [ ]:
# Make a query to find all Persons in the database
address = session.query(Address).all()
for a in address:
    print(a.street_name)



# Return the first Person from all Persons in the database
person = session.query(Person).first()
print(person.name)
# Find all Address whose person field is pointing to the person object
session.query(Address).filter(Address.person == person).all()

# Retrieve one Address whose person field is point to the person object
session.query(Address).filter(Address.person == person).one()

address = session.query(Address).filter(Address.person == person).one()
address.post_code